In [1]:
import numpy as np
import pandas as pd

# Prompt: 

### Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven-day period, identify which factors predict future user adoption.

Strategy:

1. Get the data
2. First segment the adopted based on business definition
3. Then use correlation matrix to see which features have best signal for adoption
4. Make new features to get better signal for adoption?

# Data

## Dictionary:
Users table with data on 12,000 users who signed up for the product in the last two years.  This table includes: 
* name: the user's name
-object_id: the user's id
-email: email address 
-creation_source: how their account was created. This takes on one of 5 values:
	PERSONAL_PROJECTS: invited to join another user's personal workspace
	GUEST_INVITE: invited to an organization as a guest (limited permissions)
	ORG_INVITE: invited to an organization (as a full member)
	SIGNUP: signed up via asana.com
	SIGNUP_GOOGLE_AUTH: signed up using Google Authentication (using a Google email account for their login id)
-creation_time: when they created their account
-last_session_creation_time: unix timestamp of last login
-opted_in_to_mailing_list: whether they have opted into receiving marketing emails
-enabled_for_marketing_drip: whether they are on the regular marketing email drip
-org_id: the organization (group of users) they belong to
-invited_by_user_id: which user invited them to join (if applicable).

A usage summary user engagement table that has a row for each day that a user logged into the product. 

In [2]:
users = pd.read_csv('/kaggle/input/relax-datachallenge/takehome_users.csv', encoding='latin_1')
user_engagement = pd.read_csv('/kaggle/input/relax-datachallenge/takehome_user_engagement.csv', encoding='latin_1')

In [3]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


1. get the week for each timestamp
2. then count each week per user_id
3. filter on >=3 engagements for each week for each user as adopted

Loaded users and user engagement tables into Postgres. Got adopted users based on business definition and joined back to users. With resultant table, got user information as well as those who adopted/not adopted the product.

In [5]:
adopted_users = pd.read_csv('/kaggle/input/adopted-users/adopted_users.csv')
adopted_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id
0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316.0,2
1,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1401833283,1,1,318,4143.0,10
2,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1401363998,0,0,58,NaN,20
3,33,2014-03-11 06:29:09,Araujo Josй,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1401517749,0,0,401,79.0,33
4,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1401044707,1,0,235,NaN,42
